In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [2]:
TOP_K = 50   # change if you want more or fewer athletes

In [3]:
df = pd.read_csv("athlete_events.csv")

In [4]:
df

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [5]:
cols = ["Name","Sex","Age","Height","Weight","Sport","Event","Medal"]
df = df[cols].dropna(subset=["Age","Height","Weight","Sex","Event","Sport"])

In [6]:
df

,Name,Sex,Age,Height,Weight,Sport,Event,Medal
0,A Dijiang,M,24.0,180.0,80.0,Basketball,Basketball Men's Basketball,NaN
1,A Lamusi,M,23.0,170.0,60.0,Judo,Judo Men's Extra-Lightweight,NaN
4,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Speed Skating,Speed Skating Women's 500 metres,NaN
5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
6,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...
271111,Andrzej ya,M,29.0,179.0,89.0,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,Piotr ya,M,27.0,176.0,59.0,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,Piotr ya,M,27.0,176.0,59.0,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Bobsleigh,Bobsleigh Men's Four,NaN


In [7]:
medal_map = {"Gold":3, "Silver":2, "Bronze":1}
df["MedalScore"] = df["Medal"].map(medal_map).fillna(0)

In [8]:
df["EventMean"] = df.groupby("Event")["MedalScore"].transform("mean")
df["EventStd"] = df.groupby("Event")["MedalScore"].transform("std")


In [9]:
df

,Name,Sex,Age,Height,Weight,Sport,Event,Medal,MedalScore,EventMean,EventStd
0,A Dijiang,M,24.0,180.0,80.0,Basketball,Basketball Men's Basketball,NaN,0.0,0.515644,0.987451
1,A Lamusi,M,23.0,170.0,60.0,Judo,Judo Men's Extra-Lightweight,NaN,0.0,0.222603,0.648898
4,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Speed Skating,Speed Skating Women's 500 metres,NaN,0.0,0.205479,0.665438
5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Speed Skating,"Speed Skating Women's 1,000 metres",NaN,0.0,0.186695,0.630340
6,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Speed Skating,Speed Skating Women's 500 metres,NaN,0.0,0.205479,0.665438
...,...,...,...,...,...,...,...,...,...,...,...
271111,Andrzej ya,M,29.0,179.0,89.0,Luge,Luge Mixed (Men)'s Doubles,NaN,0.0,0.336032,0.825824
271112,Piotr ya,M,27.0,176.0,59.0,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN,0.0,0.100942,0.472176
271113,Piotr ya,M,27.0,176.0,59.0,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN,0.0,0.480818,0.952199
271114,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Bobsleigh,Bobsleigh Men's Four,NaN,0.0,0.263813,0.731771


In [10]:
# Remove events with no variation
df = df[~df["EventStd"].isna() & (df["EventStd"] != 0)].copy()

In [11]:
# Standardized performance
df["PerformanceStd"] = (df["MedalScore"] - df["EventMean"]) / df["EventStd"]


In [12]:
features = ["Sex","Age","Height","Weight","Sport","Event"]
X = df[features]
y = df["PerformanceStd"]

numeric = ["Age","Height","Weight"]
categorical = ["Sex","Sport","Event"]

In [13]:
# PIPELINE: PREPROCESS + MODEL
# =========================
pre = ColumnTransformer([
    ("num", StandardScaler(), numeric),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical)
])

model = Ridge(alpha=1.0)

pipe = Pipeline([
    ("prep", pre),
    ("model", model)
])

pipe.fit(X, y)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'Height', 'Weight']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Sex', 'Sport', 'Event'])])),
                ('model', Ridge())])

In [14]:
df["Expected"] = pipe.predict(X)

In [16]:
df

,Name,Sex,Age,Height,Weight,Sport,Event,Medal,MedalScore,EventMean,EventStd,PerformanceStd,Expected
0,A Dijiang,M,24.0,180.0,80.0,Basketball,Basketball Men's Basketball,NaN,0.0,0.515644,0.987451,-0.522197,-0.118622
1,A Lamusi,M,23.0,170.0,60.0,Judo,Judo Men's Extra-Lightweight,NaN,0.0,0.222603,0.648898,-0.343048,0.007265
4,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Speed Skating,Speed Skating Women's 500 metres,NaN,0.0,0.205479,0.665438,-0.308788,0.147772
5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Speed Skating,"Speed Skating Women's 1,000 metres",NaN,0.0,0.186695,0.630340,-0.296182,0.146374
6,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Speed Skating,Speed Skating Women's 500 metres,NaN,0.0,0.205479,0.665438,-0.308788,0.161581
...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,Andrzej ya,M,29.0,179.0,89.0,Luge,Luge Mixed (Men)'s Doubles,NaN,0.0,0.336032,0.825824,-0.406906,0.058944
271112,Piotr ya,M,27.0,176.0,59.0,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN,0.0,0.100942,0.472176,-0.213781,-0.018800
271113,Piotr ya,M,27.0,176.0,59.0,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN,0.0,0.480818,0.952199,-0.504956,-0.012957
271114,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Bobsleigh,Bobsleigh Men's Four,NaN,0.0,0.263813,0.731771,-0.360513,0.043160


In [17]:
# OVERPERFORMANCE INDEX (OPI)
residual = df["PerformanceStd"] - df["Expected"]
res_sigma = residual.std()

In [18]:
df["OPI"] = residual / res_sigma   # standardize residuals

In [19]:
df["Gap"] = df["PerformanceStd"] - df["Expected"]

In [ ]:
# RANK ATHLETES
# =========================
ranked = (
    df[["Name","Sport","Event","PerformanceStd","Expected","Gap","OPI"]]
    .sort_values("OPI", ascending=False)
    .copy()
)
